In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domantas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/domantas/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/domantas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/domantas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
month = "january"
df = pd.read_csv("../Datasets/full_data_january.csv")

In [3]:
top = 2500
words_frequency = {}
for category in set(df['main_category'].values):
    print(category)
    all_words = []
    for row in df[df['main_category'] == category]['tokenized_words'].tolist():
        for word in ast.literal_eval(row):
            all_words.append(word)
    most_common = nltk.FreqDist(w for w in all_words).most_common(top)
    words_frequency[category] = most_common

Arts_and_Entertainment
Travel
Games
Science
Business_and_Industry
Books_and_Literature
Career_and_Education
Adult
Reference
Beauty_and_Fitness
People_and_Society
Home_and_Garden
Health
Finance
Autos_and_Vehicles
Internet_and_Telecom
Sports
Shopping
Food_and_Drink
Pets_and_Animals
Computer_and_Electronics
Law_and_Government
News_and_Media
Recreation_and_Hobbies
Gambling


In [4]:
for category in set(df['main_category'].values):
    words_frequency[category] = [word for word, number in words_frequency[category]]

In [5]:
from collections import Counter

features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokenized_words'])).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

In [6]:
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Dataset length: ', df.shape[0])
print()

from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Dataset length: ', df.shape[0])

/home/domantas/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/domantas/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression
Score:  0.8547826086956521
Top:  2500
Dataset length:  10452

SVM
Score:  0.7884057971014493
Top:  2500
Dataset length:  10452


/home/domantas/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [9]:
clf

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [10]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
# Save models
from sklearn.externals import joblib
filename = "../Models/{}/LR_model_{}.joblib".format(month.title(), month)
if not os.path.isfile(filename):
    joblib.dump(lr, filename)
    
filename = "../Models/{}/LSVM_model_{}.joblib".format(month.title(), month)
if not os.path.isfile(filename):
    joblib.dump(clf, filename)

import pickle
words_filename = "../Models/{}/word_frequency_{}.picle".format(month.title(), month)
if not os.path.isfile(words_filename):
    pickle_out = open(words_filename,"wb")
    pickle.dump(words_frequency, pickle_out)
    pickle_out.close()

In [ ]:
import pickle
pickle_in = open("../Models/{}/word_frequency_{}.picle".format(month.title(), month),"rb")
words_frequency_2 = pickle.load(pickle_in)

In [ ]:
import copy
temp = copy.deepcopy(words_frequency)

In [ ]:
words_frequency = copy.deepcopy(temp)

In [ ]:
# Remove chunk words
# from math import floor
# words = []
# for category in words_frequency.keys():
#     words.extend(words_frequency[category][0:20])
# words_counter = Counter(words)
# chunk_words = [x for x in words_counter if words_counter[x] >= 12]
# words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 12}
# for cat in words_frequency.keys():
#     words_frequency[cat] = [word for word in words_frequency[cat] if word not in chunk_words]

In [8]:
df_custom = pd.read_csv("../Datasets/websites_custom.csv")[['URL', 'Category']]
df_custom = df_custom[df_custom['URL'].notnull()][:20]
df_custom['Weight_model'] = ''
df_custom['lr_normal'] = ''
df_custom['lr_max'] = ''

In [9]:
top = 2500
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
row_counter = 0 
for row_id, row in df_custom.iterrows():
    row_counter += 1
    try:
        url = row['URL']
        req = urllib.request.Request(url, headers=hdr)
        html = urlopen(req, timeout=15).read()
        # html = urlopen(url, timeout=15).read()
        soup = BeautifulSoup(html, "html.parser")
        [tag.decompose() for tag in soup("script")]
        [tag.decompose() for tag in soup("style")]
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        # Tokenize text
        tokens = [token.lower() for token in toker.tokenize(text)]
        # Remove stopwords
        wnl = WordNetLemmatizer()
        tokens = [token.lower() for token in toker.tokenize(text)]
        tokens_stopwords = [w.lower() for w in tokens if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist]
        tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_stopwords]
        
        from collections import Counter
        counter = 0
        features_pred = np.zeros(top * len(words_frequency)).reshape(len(words_frequency), top)
        c = [word for word, word_count in Counter(tokens_lemmatize).most_common(top)]
        for category in words_frequency.keys():
            for word in c:
                if word in words_frequency[category]:
                    features_pred[counter][words_frequency[category].index(word)] = 1
            counter+=1

        category_weight = []
        for i in features_pred:
            weight_cof = np.where(i == 1)[0]
            weight_sum = 0
            for cof in weight_cof:
                weight_sum += top - cof
            category_weight.append(weight_sum)

        cat_index = category_weight.index(max(category_weight))
        category = list(words_frequency.keys())[cat_index]
        feature = features_pred[cat_index].reshape(-1, top)
        print("url: {} . {} / {}".format(url, row_counter, len(df_custom)))
        print('Category: ', row['Category'])
        print("My model: ",category)
        prediction = lr.predict(feature)
        print("LR normal: ", list(words_frequency.keys())[int(prediction[0])])
        df_custom.at[row_id, 'Weight_model'] = category
        df_custom.at[row_id, 'lr_normal'] = list(words_frequency.keys())[int(prediction[0])]
    except:
        print("{} - Failed. {} / {}".format(row['URL'], row_counter, len(df_custom)))
        continue

url: http://www.bbc.com/ . 1 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://www.telegraph.co.uk/news/ . 2 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://news.sky.com/world . 3 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://www.thesun.co.uk/news/ . 4 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://www.dailystar.co.uk/news . 5 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://www.express.co.uk/news . 6 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://www.theguardian.com/world . 7 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://globalnews.ca/world/ . 8 / 20
Category:  News_and_Media
My model:  News_and_Media
LR normal:  News_and_Media
url: https://edit

In [ ]:
df_custom = df_custom[df_custom['Weight_model'] != '']
model_acc = len(df_custom[df_custom['Weight_model'] == df_custom['Category']]) / len(df_custom) * 100
lr_acc = len(df_custom[df_custom['lr_normal'] == df_custom['Category']]) / len(df_custom) * 100
print("My model accuracy: {}% | {} / {}".format(model_acc, len(df_custom[df_custom['Weight_model'] == df_custom['Category']]), len(df_custom)))
print("Logistic regression accuracy: {}% | {} / {}".format(lr_acc, len(df_custom[df_custom['lr_normal'] == df_custom['Category']]), len(df_custom)))

In [ ]:
for category in list(set(df_custom['Category'].values)):
    print(category)
    df2 = df_custom[df_custom['Category'] == category]
    result_w = len(df2[df2['Weight_model'] == category])
    result_l = len(df2[df2['lr_normal'] == category])   
    print("Model: {} / {} : {:.2f}%".format(result_w, len(df2), result_w / len(df2) * 100))
    print("lr: {} / {} : {:.2f}%".format(result_l, len(df2), result_l / len(df2) * 100))
    print("*"*50)